# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-16 02:46:51] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=36092, nccl_port=None, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=10

[2025-07-16 02:46:51] Inferred chat template from model path: llama-2


[2025-07-16 02:47:02] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 02:47:02] Init torch distributed begin.


[2025-07-16 02:47:02] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 02:47:04] Load weight begin. avail mem=61.98 GB


...

[2025-07-16 02:47:07] Init torch distributed begin.
[2025-07-16 02:47:07] Init torch distributed ends. mem usage=0.00 GB
[2025-07-16 02:47:07] Load weight begin. avail mem=38.65 GB
[2025-07-16 02:47:07] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.18s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.18s/it]

[2025-07-16 02:47:09] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-07-16 02:47:09] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-16 02:47:09] Memory pool end. avail mem=37.40 GB


[2025-07-16 02:47:09] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.12 GB
[2025-07-16 02:47:09] INFO:     Started server process [2477176]
[2025-07-16 02:47:09] INFO:     Waiting for application startup.
[2025-07-16 02:47:09] INFO:     Application startup complete.
[2025-07-16 02:47:09] INFO:     Uvicorn running on http://127.0.0.1:36092 (Press CTRL+C to quit)


[2025-07-16 02:47:09] INFO:     127.0.0.1:43148 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 02:47:10] INFO:     127.0.0.1:43154 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 02:47:10] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:47:10.584100


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-16 02:48:14] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-16T02:48:14.099494


[2025-07-16 02:48:42] INFO:     127.0.0.1:43170 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 02:48:42] The server is fired up and ready to roll!


[2025-07-16 02:48:46] INFO:     127.0.0.1:43186 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-16 02:48:46] Child process unexpectedly failed with exitcode=9. pid=2477988


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-16 02:48:58] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=37156, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8026

[2025-07-16 02:48:58] Inferred chat template from model path: llama-2


[2025-07-16 02:49:09] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 02:49:09] Init torch distributed begin.


[2025-07-16 02:49:10] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 02:49:11] Load weight begin. avail mem=57.53 GB


...

[2025-07-16 02:49:15] KV Cache is allocated. #tokens: 20480, K size: 5.00 GB, V size: 5.00 GB
[2025-07-16 02:49:15] Memory pool end. avail mem=16.62 GB


[2025-07-16 02:49:16] Init torch distributed begin.
[2025-07-16 02:49:16] Init torch distributed ends. mem usage=0.00 GB
[2025-07-16 02:49:16] Load weight begin. avail mem=15.80 GB
[2025-07-16 02:49:16] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]

[2025-07-16 02:49:17] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=14.57 GB, mem usage=1.23 GB.
[2025-07-16 02:49:17] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-16 02:49:17] Memory pool end. avail mem=14.38 GB


[2025-07-16 02:49:18] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=14.36 GB
[2025-07-16 02:49:18] INFO:     Started server process [2484756]
[2025-07-16 02:49:18] INFO:     Waiting for application startup.
[2025-07-16 02:49:18] INFO:     Application startup complete.
[2025-07-16 02:49:18] INFO:     Uvicorn running on http://127.0.0.1:37156 (Press CTRL+C to quit)
[2025-07-16 02:49:18] INFO:     127.0.0.1:48770 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 02:49:19] INFO:     127.0.0.1:44486 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 02:49:19] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:49:19.210462


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-16 02:49:23] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-16T02:49:23.488640


[2025-07-16 02:49:25] INFO:     127.0.0.1:44492 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 02:49:25] The server is fired up and ready to roll!
[2025-07-16 02:49:25] INFO:     127.0.0.1:44502 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-16 02:49:25] Child process unexpectedly failed with exitcode=9. pid=2485342
[2025-07-16 02:49:25] Child process unexpectedly failed with exitcode=9. pid=2485205


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-16 02:49:39] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=33123, nccl_port=None, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=Fa

[2025-07-16 02:49:50] Casting torch.bfloat16 to torch.float16.


[2025-07-16 02:49:51] Casting torch.bfloat16 to torch.float16.
[2025-07-16 02:49:51] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 02:49:51] Init torch distributed begin.


[2025-07-16 02:49:52] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 02:49:54] Load weight begin. avail mem=78.50 GB


...

[2025-07-16 02:50:11] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-07-16 02:50:12] INFO:     Started server process [2487091]
[2025-07-16 02:50:12] INFO:     Waiting for application startup.
[2025-07-16 02:50:12] INFO:     Application startup complete.
[2025-07-16 02:50:12] INFO:     Uvicorn running on http://127.0.0.1:33123 (Press CTRL+C to quit)


[2025-07-16 02:50:12] INFO:     127.0.0.1:56886 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 02:50:13] INFO:     127.0.0.1:56894 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 02:50:13] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:50:13.314108


[2025-07-16 02:50:15] INFO:     127.0.0.1:56908 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 02:50:15] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-16 02:50:17] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:50:17.763446


[2025-07-16 02:50:18] INFO:     127.0.0.1:56912 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-16 02:50:18] Child process unexpectedly failed with exitcode=9. pid=2487850


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-16 02:50:28] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='127.0.0.1', port=31281, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-07-16 02:50:40] Casting torch.bfloat16 to torch.float16.


[2025-07-16 02:50:40] Casting torch.bfloat16 to torch.float16.
[2025-07-16 02:50:40] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 02:50:40] Init torch distributed begin.


[2025-07-16 02:50:41] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 02:50:42] Load weight begin. avail mem=78.50 GB


...

[2025-07-16 02:50:58] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-16 02:50:58] Memory pool end. avail mem=57.88 GB


[2025-07-16 02:50:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-16 02:51:00] INFO:     Started server process [2489685]
[2025-07-16 02:51:00] INFO:     Waiting for application startup.
[2025-07-16 02:51:00] INFO:     Application startup complete.
[2025-07-16 02:51:00] INFO:     Uvicorn running on http://127.0.0.1:31281 (Press CTRL+C to quit)
[2025-07-16 02:51:00] INFO:     127.0.0.1:35934 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 02:51:01] INFO:     127.0.0.1:35950 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 02:51:01] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:51:01.566499


[2025-07-16 02:51:04] INFO:     127.0.0.1:35962 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 02:51:04] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-16 02:51:05] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:51:05.647453


[2025-07-16 02:51:05] INFO:     127.0.0.1:35976 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-16 02:51:05] Child process unexpectedly failed with exitcode=9. pid=2490102


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-16 02:51:17] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=38723, nccl_port=None, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=163877352, constrained_json_whi

[2025-07-16 02:51:28] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 02:51:28] Init torch distributed begin.


[2025-07-16 02:51:28] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 02:51:29] Load weight begin. avail mem=78.50 GB


[2025-07-16 02:51:29] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  8.96it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  8.51it/s]

[2025-07-16 02:51:34] Load weight end. type=MiMoMTP, dtype=torch.bfloat16, avail mem=57.83 GB, mem usage=2.71 GB.
[2025-07-16 02:51:34] KV Cache is allocated. #tokens: 20480, K size: 1.41 GB, V size: 1.41 GB
[2025-07-16 02:51:34] Memory pool end. avail mem=54.88 GB


[2025-07-16 02:51:34] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-16 02:51:35] INFO:     Started server process [2491779]
[2025-07-16 02:51:35] INFO:     Waiting for application startup.
[2025-07-16 02:51:35] INFO:     Application startup complete.
[2025-07-16 02:51:35] INFO:     Uvicorn running on http://0.0.0.0:38723 (Press CTRL+C to quit)


[2025-07-16 02:51:36] INFO:     127.0.0.1:50030 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-16 02:51:36] INFO:     127.0.0.1:50046 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 02:51:36] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:51:36.054333


[2025-07-16 02:51:37] INFO:     127.0.0.1:50052 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 02:51:37] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-16 02:51:41] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T02:51:41.038412


[2025-07-16 02:51:41] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, accept len: 1.69, cuda graph: False, gen throughput (token/s): 9.65, #queue-req: 0, timestamp: 2025-07-16T02:51:41.667344


[2025-07-16 02:51:42] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, accept len: 1.68, cuda graph: False, gen throughput (token/s): 101.34, #queue-req: 0, timestamp: 2025-07-16T02:51:42.328507


[2025-07-16 02:51:42] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 104.85, #queue-req: 0, timestamp: 2025-07-16T02:51:42.986618


[2025-07-16 02:51:43] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, accept len: 1.77, cuda graph: False, gen throughput (token/s): 105.06, #queue-req: 0, timestamp: 2025-07-16T02:51:43.662404


[2025-07-16 02:51:44] Decode batch. #running-req: 1, #token: 361, token usage: 0.02, accept len: 1.65, cuda graph: False, gen throughput (token/s): 98.64, #queue-req: 0, timestamp: 2025-07-16T02:51:44.331507


[2025-07-16 02:51:44] INFO:     127.0.0.1:48748 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-16 02:51:44] Child process unexpectedly failed with exitcode=9. pid=2492086
[2025-07-16 02:51:44] Child process unexpectedly failed with exitcode=9. pid=2491949


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).